In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

In [2]:
def init_driver():
    driver = webdriver.Chrome()
    driver.wait = WebDriverWait(driver, 3)
    return driver

In [3]:
def lookup(driver):
    post_dict =  {'link':[], 'title':[], 'stats':[], 'last_post_date':[]}
    driver.get('http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat')
    driver.find_element_by_xpath('''//*[@id="yui-gen11"]''').click()
    counter1 = 0
    counter2 = 0
    counter3 = 0
    page_number = 1

    while page_number==1:
        try:
            link = driver.find_element_by_link_text(str(page_number))
        except NoSuchElementException:
            break

        posts = driver.find_elements_by_class_name('threadtitle')
        for post in posts:
            #print(post.find_element_by_css_selector('a').get_attribute('href'))
            post_dict['link'].append(post.find_element_by_css_selector('a').get_attribute('href'))
            #print(post.text)
            post_dict['title'].append(post.text)
            counter1 += 1
        print(counter1)

        #stats_h = driver.find_elements_by_xpath('''//*[@class='threadstats td']''')
        post_dict['stats'].append(driver.find_elements_by_xpath('''//*[@class='threadstats td']'''))
        counter2 += 1
        stats = driver.find_elements_by_xpath('''//*[@class='threadstats td alt']''')
        for stat in stats:
            #print(stat.text)
            post_dict['stats'].append(stat.text)
            counter2 += 1
        print(counter2)

        postdates = driver.find_elements_by_xpath('''//*[@class='threadlastpost td']''')
        for postdate in postdates:
            #print(postdate.text)
            post_dict['last_post_date'].append(postdate.text.split(','))
            counter3 += 1
        print(counter3)

        link.click()
        print(driver.current_url)
        page_number += 1

    return post_dict

In [4]:
driver = init_driver()
data = lookup(driver)

21
21
21
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat


In [5]:
data = pd.DataFrame.from_dict(data)

In [6]:
data.head()

,last_post_date,link,stats,title
0,[LAST POST BY],http://www.f150ecoboost.net/forum/68-2016-ford...,[<selenium.webdriver.remote.webelement.WebElem...,TITLE / THREAD STARTER
1,"[gsxrdoug\nToday, 07:28 PM]",http://www.f150ecoboost.net/forum/68-2016-ford...,Replies: 16\nViews: 415,Lets hear from the happy 2.7 owners that have ...
2,"[ADR86\nYesterday, 02:28 PM]",http://www.f150ecoboost.net/forum/68-2016-ford...,Replies: 1\nViews: 57,Air Conditioning hiccup
3,"[Tdog02\n08-20-2017, 10:55 AM]",http://www.f150ecoboost.net/forum/68-2016-ford...,"Replies: 92\nViews: 19,984",2.7 EB long term updates (normal guy)
4,"[Quinnford\n08-17-2017, 12:19 PM]",http://www.f150ecoboost.net/forum/68-2016-ford...,Replies: 3\nViews: 163,Foam Cover Purpose?
